# Start your work here :)
# Happy coding ~

In [1]:
import os
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import SciServer.CasJobs as cj

print (sp.__version__)
cpath = '/home/idies/workspace/Storage/AS_171_205_2021/AS_171_205_2021-Course/data/'

1.6.0


## Problem 10.1 ##

* Use COVID NYT Database on SciServer to perform the following statistical analysis from a Jupyter notebook

### 10.1.1 ###

* Select 200 counties in US with highest and lowest population densities. 
* Extract total population in both categories (sum of highest density and sum of lowest)
* Debug Queries

In [2]:
TOP_QUERY = 'SELECT TOP 200 * FROM censusData WHERE county is not null ORDER BY Popdensity DESC'
BOTTOM_QUERY = 'SELECT TOP 200 * FROM censusData WHERE county is not null ORDER BY Popdensity ASC'
ALL_QUERY = 'SELECT * FROM censusData WHERE county is not null'

TOP_200_COUNTIES = '(SELECT TOP 200 county FROM censusData WHERE county is not null ORDER BY Popdensity DESC)'
BOTTOM_200_COUNTIES = '(SELECT TOP 200 county FROM censusData WHERE county is not null ORDER BY Popdensity ASC)'
def getTop200():
    return cj.executeQuery(TOP_QUERY,"COVIDNYT")

def getBottom200():
    return cj.executeQuery(BOTTOM_QUERY,"COVIDNYT")

def allCounties():
    return cj.executeQuery(ALL_QUERY,"COVIDNYT")

top200 = getTop200() 
bottom200 = getBottom200() 
allCounties = allCounties()

top200Pop = np.sum(top200['Population'])
bottom200Pop = np.sum(bottom200['Population'])

totalPop = np.sum(allCounties['Population'])

print("\nTop 200 total population: " + str(top200Pop))
print("\nBottom 200 total population: " + str(bottom200Pop))
print("\nTotal population: " + str(totalPop))




Top 200 total population: 126557015

Bottom 200 total population: 799810

Total population: 312471327


### 10.1.2 ###

* Print 4 different results
* 1--> Max infections in top 200
* 2--> Max deaths in top 200
* 3--> Max infections in bottom 200
* 4--> Max deaths in bottom 200

In [3]:
def getTop():
    # get the time series for the top 200 counties
    sql="""
    select date, sum(cases) cases, sum(deaths) deaths
    from StatsC
    where fips in (select top 200 fips from censusData order by popdensity desc)
    group by date 
    order by date
    """
    return cj.executeQuery(sql,"COVIDNYT")
def getBot():
    sql="""
    select date, sum(cases) cases, sum(deaths) deaths
    from StatsC
    where fips in (select top 200 fips from censusData order by popdensity asc)
    group by date
    order by date
    """
    return cj.executeQuery(sql,"COVIDNYT")
top = getTop()
bot = getBot()
#mi = Max Infections
#md = Max Deaths
mit = np.argmax(top['cases'])
mdt = np.argmax(top['deaths'])
mib = np.argmax(bot['cases'])
mdb = np.argmax(bot['deaths'])
dft = top.iloc[mit]
dfb = bot.iloc[mib]

In [17]:
def colored(r, g, b, text):
    return "\033[38;2;{};{};{}m{} \033[38;2;255;255;255m".format(r, g, b, text)

print(colored(255, 0, 0, "Top 200 Stats: "))
print(colored(0, 0, 0,dft))
print(colored(255, 0, 0,"Bottom 200 Stats: "))
print(colored(0, 0 , 0, dfb))
print(colored(29, 219, 29, "Max Infections") + colored(255,0,0, "top") + colored(29, 219, 29, "200: "))
print(colored(0 , 0 , 0, top.iloc[mit]['cases']))
print(colored(11, 28, 217, "Max Deaths") + colored(255,0,0, "top") + colored(11, 28, 217, "200: "))
print(colored(0 , 0 , 0, top.iloc[mdt]['deaths']))
print(colored(29, 219, 29, "Max Infections") + colored(255,0,0, "bottom") + colored(29, 219, 29, "200: "))
print(colored(0 , 0 , 0, bot.iloc[mib]['cases']))
print(colored(11, 28, 217, "Max Deaths") + colored(255,0,0, "bottom") + colored(11, 28, 217,"200: "))
print(colored(0 , 0 , 0, bot.iloc[mdb]['deaths']))

Top 200 Stats:  
date      4/18/2021 12:00:00 AM
cases                  12845561
deaths                   241091
Name: 450, dtype: object 
Bottom 200 Stats:  
date      4/18/2021 12:00:00 AM
cases                     69162
deaths                     1256
Name: 395, dtype: object 
Max Infections top 200:  
12845561 
Max Deaths top 200:  
241091 
Max Infections bottom 200:  
69162 
Max Deaths bottom 200:  
1256 


### 10.1.3 ###

* Normalize above values to 100,000 people
* Determine difference
* Stand Dev

In [35]:
normTopCases = 100000 * top.iloc[mit]['cases'] / top200Pop
normTopDeaths = 100000 * top.iloc[mdt]['deaths'] / top200Pop

normBotCases = 100000 * bot.iloc[mib]['cases'] / bottom200Pop
normBotDeaths = 1e5 * bot.iloc[mdb]['deaths'] / bottom200Pop
print("Top 200 Max (cases, deaths) normalized: " + str((int(normTopCases), int(normTopDeaths))))
print("Bottom 200 Max (cases,deaths) normalized: " + str((int(normBotCases), int(normBotDeaths))))
stdTopCases = np.std(top['cases']) 
stdTopDeaths = np.std(top['deaths']) 

stdBottomCases = np.std(bot['cases']) 
stdBottomDeaths = np.std(bot['deaths']) 

tCases = np.abs(normTopCases - normBotCases) / np.sqrt((stdTopCases**2 + stdBottomCases**2))
tDeaths = np.abs(normTopDeaths - normBotDeaths) / np.sqrt((stdTopDeaths**2 + stdBottomDeaths**2))

print("\nT-tests")
print(colored(255, 146, 21,"Cases: "), (colored(0, 0 , 0, tCases)))
print(colored(100, 45, 212,"Deaths: "), (colored(0, 0 , 0, tDeaths)))

Top 200 Max (cases, deaths) normalized: (10150, 190)
Bottom 200 Max (cases,deaths) normalized: (8647, 157)

T-tests
Cases:   0.00036041234530670695 
Deaths:   0.00046677757423528945 
